In [2]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0  # ✅ Import EfficientNetB0

print("Start")

# Set paths for extracted frames
frames_dir = "D:/dataset/archive (1)"

# Set parameters
batch_size = 16
img_height = 224
img_width = 224
epochs = 30

# Create a data generator for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Create generators for training and validation
train_generator = train_datagen.flow_from_directory(
    directory=frames_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

print("Training data generator created.")

validation_generator = train_datagen.flow_from_directory(
    directory=frames_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

print("Validation data generator created.")

# ✅ Load EfficientNetB0 as base model
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)

# Freeze the base model
base_model.trainable = False

print("EfficientNetB0 base model loaded and frozen.")

# Create the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Model compiled.")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")

# Measure training time
start_time = time.time()

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    verbose=1
)

# Measure end time
end_time = time.time()
total_time = end_time - start_time

# Save the model
model.save('violence_detection_model_efficientnetb0.h5')

print("Model training complete and saved as 'violence_detection_model_efficientnetb0.h5'.")
print(f"Total training time: {total_time:.2f} seconds.")


Start
Found 3857 images belonging to 2 classes.
Training data generator created.
Found 964 images belonging to 2 classes.
Validation data generator created.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 49s 3us/step
EfficientNetB0 base model loaded and frozen.
Model compiled.
Training samples: 3857
Validation samples: 964
Epoch 1/30
242/242 ━━━━━━━━━━━━━━━━━━━━ 104s 401ms/step - accuracy: 0.7754 - loss: 0.5370 - val_accuracy: 0.7822 - val_loss: 0.5268
Epoch 2/30
242/242 ━━━━━━━━━━━━━━━━━━━━ 76s 312ms/step - accuracy: 0.7819 - loss: 0.5296 - val_accuracy: 0.7822 - val_loss: 0.5253
Epoch 3/30
242/242 ━━━━━━━━━━━━━━━━━━━━ 73s 301ms/step - accuracy: 0.7796 - loss: 0.5315 - val_accuracy: 0.7822 - val_loss: 0.5278
Epoch 4/30
242/242 ━━━━━━━━━━━━━━━━━━━━ 75s 309ms/step - accuracy: 0.7846 - loss: 0.5258 - val_accuracy: 0.7822 - val_loss: 0.5250
Epoch 5/30
242/242 ━━━━━━━━━━━━━━━━━━━━ 76s 315ms/step - accuracy: 0.7792 - loss: 0.5308 - val_accuracy: 0.7822 - val_loss: 0.5239
Epoch 6/30
242/242 ━━━━━━━━

Model training complete and saved as 'violence_detection_model_efficientnetb0.h5'.
Total training time: 2342.83 seconds.


In [1]:
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Start")

# Dataset paths and parameters
frames_dir = "D:/dataset/archive (1)"
batch_size = 16
img_height = 224
img_width = 224
additional_epochs = 270  # How many more epochs you want to train

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=frames_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory=frames_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

print("Data generators ready.")

# ✅ Load the previously trained model
model = load_model('violence_detection_model_efficientnetb0.h5')
print("Loaded model from previous training.")

# If you want to fine-tune base model too (optional)
# Uncomment below to unfreeze and train full model
# model.layers[0].trainable = True
# print("Base model unfrozen for fine-tuning.")

# Re-compile model if you change `trainable` state
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Model recompiled.")

# Train again
start_time = time.time()

history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=additional_epochs,
    verbose=1
)

end_time = time.time()
print(f"Additional training time: {end_time - start_time:.2f} seconds")

# Save updated model
model.save('violence_detection_model_efficientnetb0_finetuned.h5')
print("Model saved after continued training.")


Start
Found 3857 images belonging to 2 classes.
Found 964 images belonging to 2 classes.
Data generators ready.


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Loaded model from previous training.
Model recompiled.
Epoch 1/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 109s 421ms/step - accuracy: 0.7833 - loss: 0.5441 - val_accuracy: 0.7822 - val_loss: 0.5248
Epoch 2/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 80s 332ms/step - accuracy: 0.7781 - loss: 0.5309 - val_accuracy: 0.7822 - val_loss: 0.5242
Epoch 3/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 77s 317ms/step - accuracy: 0.7752 - loss: 0.5353 - val_accuracy: 0.7822 - val_loss: 0.5285
Epoch 4/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 81s 334ms/step - accuracy: 0.7802 - loss: 0.5294 - val_accuracy: 0.7822 - val_loss: 0.5260
Epoch 5/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 82s 338ms/step - accuracy: 0.7900 - loss: 0.5147 - val_accuracy: 0.7822 - val_loss: 0.5242
Epoch 6/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 81s 334ms/step - accuracy: 0.7753 - loss: 0.5340 - val_accuracy: 0.7822 - val_loss: 0.5278
Epoch 7/270
242/242 ━━━━━━━━━━━━━━━━━━━━ 78s 321ms/step - accuracy: 0.7962 - loss: 0.5071 - val_accuracy: 0.7822 - val_loss: 0.5248
Epoch 8/270
242/242 

Additional training time: 20747.13 seconds


TypeError: cannot pickle 'module' object

In [3]:
model.save('violence_detection_model_efficientnetb0_finetuned.keras')
print("done")

done
